# Rescaled Range Analysis

In [2]:
import pandas as pd
import numpy as np

In [4]:
df_spot = pd.read_pickle('overall.pkl')

In [6]:
df_spot

,time_period_start,time_period_end,time_open,time_close,price_open,price_high,price_low,price_close,volume_traded,trades_count,ticker
100,2020-10-27T04:00:00.0000000Z,2020-10-27T05:00:00.0000000Z,2020-10-27T04:00:00.1790000Z,2020-10-27T04:59:59.7730000Z,13083.24,13125.00,13083.24,13124.60,1100.816862,15977,BINANCE_SPOT_BTC_USDT
101,2020-10-27T05:00:00.0000000Z,2020-10-27T06:00:00.0000000Z,2020-10-27T05:00:01.0350000Z,2020-10-27T05:59:59.8560000Z,13124.61,13144.49,13080.24,13088.65,1447.336487,19319,BINANCE_SPOT_BTC_USDT
102,2020-10-27T06:00:00.0000000Z,2020-10-27T07:00:00.0000000Z,2020-10-27T06:00:00.6080000Z,2020-10-27T06:59:58.3880000Z,13088.66,13106.09,13061.01,13106.09,1570.109937,19349,BINANCE_SPOT_BTC_USDT
103,2020-10-27T07:00:00.0000000Z,2020-10-27T08:00:00.0000000Z,2020-10-27T07:00:00.3400000Z,2020-10-27T07:59:59.9890000Z,13106.08,13160.00,13086.97,13144.58,2602.515018,26664,BINANCE_SPOT_BTC_USDT
104,2020-10-27T08:00:00.0000000Z,2020-10-27T09:00:00.0000000Z,2020-10-27T08:00:00.1290000Z,2020-10-27T08:59:59.6000000Z,13144.59,13170.21,13125.00,13141.56,2226.309279,33975,BINANCE_SPOT_BTC_USDT
...,...,...,...,...,...,...,...,...,...,...,...
95,2022-08-31T03:00:00.0000000Z,2022-08-31T04:00:00.0000000Z,2022-08-31T03:00:00.7890000Z,2022-08-31T03:59:56.8950000Z,19.78,19.97,19.77,19.81,86885.360000,3834,BINANCE_SPOT_AVAX_USDT
96,2022-08-31T04:00:00.0000000Z,2022-08-31T05:00:00.0000000Z,2022-08-31T04:00:00.2050000Z,2022-08-31T04:59:52.6790000Z,19.80,19.89,19.69,19.70,102132.780000,3445,BINANCE_SPOT_AVAX_USDT
97,2022-08-31T05:00:00.0000000Z,2022-08-31T06:00:00.0000000Z,2022-08-31T05:00:00.9960000Z,2022-08-31T05:59:56.3010000Z,19.70,19.87,19.66,19.80,76343.660000,3193,BINANCE_SPOT_AVAX_USDT
98,2022-08-31T06:00:00.0000000Z,2022-08-31T07:00:00.0000000Z,2022-08-31T06:00:01.7450000Z,2022-08-31T06:59:55.3700000Z,19.81,20.07,19.58,19.64,280801.860000,9443,BINANCE_SPOT_AVAX_USDT


In [3]:
df_perp = pd.read_pickle('perp_overall.pkl')

#### Returns for 1 series and Hurst exponent

In [16]:
btc_usdt = df_spot.loc[df_spot['ticker'] == 'BINANCE_SPOT_BTC_USDT']
btc_usdt = btc_usdt[['time_period_start', 'price_close', 'ticker']] 
btc_usdt.reset_index(drop=True, inplace=True)
btc_usdt['time_period_start'] = pd.to_datetime(btc_usdt['time_period_start'], utc=True)

In [19]:
btc_usdt['ret'] = btc_usdt['price_close'].pct_change() 

In [24]:
res_btc_usdt = btc_usdt[btc_usdt['time_period_start'] >= '2020-11-01']

## One month worth of 1hr data

In [35]:
hurst_one_range = res_btc_usdt.head(672)
hurst_two_range_a = hurst_one_range.iloc[:336,:]
hurst_two_range_b = hurst_one_range.iloc[336:,:]

In [40]:
hurst_two_range_b.head()

,time_period_start,price_close,ticker,ret
452,2020-11-15 00:00:00+00:00,16040.58,BINANCE_SPOT_BTC_USDT,-0.001859
453,2020-11-15 01:00:00+00:00,15981.98,BINANCE_SPOT_BTC_USDT,-0.003653
454,2020-11-15 02:00:00+00:00,15984.14,BINANCE_SPOT_BTC_USDT,0.000135
455,2020-11-15 03:00:00+00:00,15929.47,BINANCE_SPOT_BTC_USDT,-0.003420
456,2020-11-15 04:00:00+00:00,15936.89,BINANCE_SPOT_BTC_USDT,0.000466


In [50]:
hurst_one_range = res_btc_usdt.head(672)
ranges = {
    'one range': hurst_one_range,
    'two ranges': np.array_split(hurst_one_range, 2),
    'four ranges': np.array_split(hurst_one_range, 4),
    'eight range': np.array_split(hurst_one_range, 8),
    '16 range': np.array_split(hurst_one_range, 16),
    '32 range': np.array_split(hurst_one_range, 32),
}

In [69]:
list(ranges.values())[1][0]['ret'].mean()

0.0004745478555720082

In [88]:
list(ranges.values())[4][7]['ret'].mean()

-0.0002325701811199108

In [78]:
list_of_means = [list(ranges.values())[0]['ret'].mean()]

#### Means calculation test

In [85]:
range_values = [2, 4, 6, 8, 16, 32]
# loop through dict
for i in range(1, len(ranges)):
    
    # loop through the range_values for each inner df
    for j in range(range_values[i]):
        # print(i,j)
        mean_value = list(ranges.values())[i][j]['ret'].mean()
        list_of_means.append(mean_value)


IndexError: list index out of range

In [52]:
ranges['32 range'][0]

,time_period_start,price_close,ticker,ret
116,2020-11-01 00:00:00+00:00,13725.95,BINANCE_SPOT_BTC_USDT,-0.004717
117,2020-11-01 01:00:00+00:00,13759.90,BINANCE_SPOT_BTC_USDT,0.002473
118,2020-11-01 02:00:00+00:00,13715.88,BINANCE_SPOT_BTC_USDT,-0.003199
119,2020-11-01 03:00:00+00:00,13690.01,BINANCE_SPOT_BTC_USDT,-0.001886
120,2020-11-01 04:00:00+00:00,13677.25,BINANCE_SPOT_BTC_USDT,-0.000932
121,2020-11-01 05:00:00+00:00,13698.23,BINANCE_SPOT_BTC_USDT,0.001534
122,2020-11-01 06:00:00+00:00,13714.56,BINANCE_SPOT_BTC_USDT,0.001192
123,2020-11-01 07:00:00+00:00,13792.51,BINANCE_SPOT_BTC_USDT,0.005684
124,2020-11-01 08:00:00+00:00,13748.62,BINANCE_SPOT_BTC_USDT,-0.003182
125,2020-11-01 09:00:00+00:00,13687.52,BINANCE_SPOT_BTC_USDT,-0.004444


In [49]:
np.array_split(hurst_one_range, 1)

[            time_period_start  price_close                 ticker       ret
 116 2020-11-01 00:00:00+00:00     13725.95  BINANCE_SPOT_BTC_USDT -0.004717
 117 2020-11-01 01:00:00+00:00     13759.90  BINANCE_SPOT_BTC_USDT  0.002473
 118 2020-11-01 02:00:00+00:00     13715.88  BINANCE_SPOT_BTC_USDT -0.003199
 119 2020-11-01 03:00:00+00:00     13690.01  BINANCE_SPOT_BTC_USDT -0.001886
 120 2020-11-01 04:00:00+00:00     13677.25  BINANCE_SPOT_BTC_USDT -0.000932
 ..                        ...          ...                    ...       ...
 783 2020-11-28 19:00:00+00:00     17814.50  BINANCE_SPOT_BTC_USDT  0.004767
 784 2020-11-28 20:00:00+00:00     17673.21  BINANCE_SPOT_BTC_USDT -0.007931
 785 2020-11-28 21:00:00+00:00     17741.05  BINANCE_SPOT_BTC_USDT  0.003839
 786 2020-11-28 22:00:00+00:00     17760.03  BINANCE_SPOT_BTC_USDT  0.001070
 787 2020-11-28 23:00:00+00:00     17719.85  BINANCE_SPOT_BTC_USDT -0.002262
 
 [672 rows x 4 columns]]